In [ ]:
import os
from openai import AzureOpenAI
import gradio as gr
from azure.cognitiveservices.speech import SpeechConfig, SpeechSynthesizer, AudioConfig

# Azure OpenAI 및 Cognitive Search 설정
endpoint = os.getenv("ENDPOINT_URL", "https://team10-eighti.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
search_endpoint = os.getenv("SEARCH_ENDPOINT", "https://team10-eighti-search.search.windows.net")
search_key = os.getenv("SEARCH_KEY", "wnalAsW6FqKRHIR6S3sUZGzNH28Lf3sBOS2ubCZsZxAzSeA205k3")
search_index = os.getenv("SEARCH_INDEX_NAME", "sign-index")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "1BajXTI5Mp0tKrs46XFGuOWjSPXKzOZSKy8e6R3qha1SVQ4lz1PFJQQJ99ALACYeBjFXJ3w3AAABACOGgPvw")

# Azure Speech Service 설정 (한국어 음성 출력)
speech_key = os.getenv("AZURE_SPEECH_KEY", "AwjVcsBAkpnrMNwkobsgJ4SSroO1GkAztrEIYp1JuMTcuKcfDR3wJQQJ99ALACYeBjFXJ3w3AAAYACOGZMDc")
speech_region = os.getenv("AZURE_SPEECH_REGION", "eastus")
speech_config = SpeechConfig(subscription=speech_key, region=speech_region)
speech_config.speech_synthesis_language = "ko-KR"  # 한국어 설정
speech_config.speech_synthesis_voice_name = "ko-KR-SunHiNeural"  # 한국어 음성
speech_synthesizer = SpeechSynthesizer(speech_config=speech_config)

# Azure OpenAI 클라이언트 초기화
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2024-05-01-preview",
)

# 채팅 함수 정의
def chat_with_openai(user_input, chat_history):
    try:
        # 대화 기록 포함 메시지 구성
        messages = [{"role": "system", "content": "너는 수화를 알려주는 전문가야"}]
        for user_msg, assistant_msg in chat_history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})
        messages.append({"role": "user", "content": user_input})

        # Azure OpenAI API 호출
        completion = client.chat.completions.create(
            model=deployment,
            messages=messages,
            max_tokens=800,
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            stream=False,
            extra_body={
                "data_sources": [{
                    "type": "azure_search",
                    "parameters": {
                        "endpoint": f"{search_endpoint}",
                        "index_name": search_index,
                        "semantic_configuration": "sign-semantic",
                        "query_type": "semantic",
                        "fields_mapping": {},
                        "in_scope": True,
                        "role_information": "너는 수화를 알려주는 전문가야",
                        "filter": None,
                        "strictness": 3,
                        "top_n_documents": 5,
                        "authentication": {
                            "type": "api_key",
                            "key": f"{search_key}"
                        }
                    }
                }]
            }
        )

        # 응답 추출 및 대화 기록 업데이트
        assistant_reply = completion.choices[0].message.content  # 속성 접근 방식으로 수정
        chat_history.append((user_input, assistant_reply))  # 튜플 형태로 추가

        # Azure Speech Service로 응답 읽기 (텍스트를 한국어 음성으로 변환)
        speech_synthesizer.speak_text_async(assistant_reply)

        return chat_history, chat_history

    except Exception as e:
        return [("Error", str(e))], chat_history

# Gradio 인터페이스 구성
with gr.Blocks() as demo:
    gr.Markdown("# Azure OpenAI + Cognitive Search + Speech 기반 수화 챗봇")
    chatbot = gr.Chatbot()
    user_input = gr.Textbox(label="Your Message", placeholder="메시지를 입력하세요...")
    clear_button = gr.Button("Clear Chat")
    
    # 대화 기록 저장
    state = gr.State([])

    # 이벤트 연결
    user_input.submit(chat_with_openai, [user_input, state], [chatbot, state])
    clear_button.click(lambda: [], None, chatbot)

# 실행
demo.launch()


In [ ]:

import os
from openai import AzureOpenAI
import gradio as gr
from azure.cognitiveservices.speech import SpeechConfig, SpeechSynthesizer, AudioConfig
import re

# Azure OpenAI 및 Cognitive Search 설정
endpoint = os.getenv("ENDPOINT_URL", "https://team10-eighti.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
search_endpoint = os.getenv("SEARCH_ENDPOINT", "https://team10-eighti-search.search.windows.net")
search_key = os.getenv("SEARCH_KEY", "wnalAsW6FqKRHIR6S3sUZGzNH28Lf3sBOS2ubCZsZxAzSeA205k3")
search_index = os.getenv("SEARCH_INDEX_NAME", "sign-index")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "1BajXTI5Mp0tKrs46XFGuOWjSPXKzOZSKy8e6R3qha1SVQ4lz1PFJQQJ99ALACYeBjFXJ3w3AAABACOGgPvw")

# Azure Speech Service 설정 (한국어 음성 출력)
speech_key = os.getenv("AZURE_SPEECH_KEY", "AwjVcsBAkpnrMNwkobsgJ4SSroO1GkAztrEIYp1JuMTcuKcfDR3wJQQJ99ALACYeBjFXJ3w3AAAYACOGZMDc")
speech_region = os.getenv("AZURE_SPEECH_REGION", "eastus")
speech_config = SpeechConfig(subscription=speech_key, region=speech_region)
speech_config.speech_synthesis_language = "ko-KR"  # 한국어 설정
speech_config.speech_synthesis_voice_name = "ko-KR-SunHiNeural"  # 한국어 음성
speech_synthesizer = SpeechSynthesizer(speech_config=speech_config)

# Azure OpenAI 클라이언트 초기화
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2024-05-01-preview",
)

# 링크 추출 함수
def extract_links(text):
    image_links = []
    video_links = []
    for line in text.split("\n"):
        if line.endswith(".jpg") or line.endswith(".png"):
            image_links.append(line.strip())
        elif line.endswith(".mp4"):
            video_links.append(line.strip())
    return image_links, video_links


# 채팅 함수 정의
def chat_with_openai(user_input, chat_history):
    try:
        # 대화 기록 포함 메시지 구성
        messages = [{"role": "system", "content": "너는 수화를 알려주는 전문가야"}]
        for user_msg, assistant_msg in chat_history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})
        messages.append({"role": "user", "content": user_input})

        # Azure OpenAI API 호출
        completion = client.chat.completions.create(
            model=deployment,
            messages=messages,
            max_tokens=800,
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            stream=False,
            extra_body={
                "data_sources": [{
                    "type": "azure_search",
                    "parameters": {
                        "endpoint": f"{search_endpoint}",
                        "index_name": search_index,
                        "semantic_configuration": "sign-semantic",
                        "query_type": "semantic",
                        "fields_mapping": {},
                        "in_scope": True,
                        "role_information": "너는 수화를 알려주는 전문가야",
                        "filter": None,
                        "strictness": 3,
                        "top_n_documents": 5,
                        "authentication": {
                            "type": "api_key",
                            "key": f"{search_key}"
                        }
                    }
                }]
            }
        )

        # 응답 추출 및 대화 기록 업데이트
        assistant_reply = completion.choices[0].message.context['citations'][0]['content']  # 속성 접근 방식으로 수정
        
        # 이미지 및 동영상 링크 추출
        image_links, video_links = extract_links(assistant_reply)
        
        chat_history.append((user_input, assistant_reply))  # 튜플 형태로 추가

        # 음성 출력 (링크 제외 텍스트만 읽기)
        speech_text = re.sub(r"http\S+", "", assistant_reply).strip()
        speech_synthesizer.speak_text_async(speech_text)

        # 반환: 대화 기록, 첫 번째 이미지 링크, 첫 번째 동영상 링크
        return chat_history, image_links[0] if image_links else None, video_links[0] if video_links else None


    except Exception as e:
        return [("Error", str(e))], chat_history

# Gradio 인터페이스 구성
with gr.Blocks() as demo:
    gr.Markdown("# Azure OpenAI + Cognitive Search + Speech 기반 수화 챗봇")

    with gr.Row():
        with gr.Column(scale=3):
            chatbot = gr.Chatbot()
            user_input = gr.Textbox(label="Your Message", placeholder="메시지를 입력하세요...")
            clear_button = gr.Button("Clear Chat")
        
        with gr.Column(scale=1):
            video_display = gr.Video(label="관련 동영상")
            image_display = gr.Image(label="관련 이미지")
            
    state = gr.State([])

    # 이벤트 연결
    user_input.submit(chat_with_openai, [user_input, state], [chatbot, image_display, video_display])
    clear_button.click(lambda: ([], None, None), None, [chatbot, image_display, video_display])

demo.launch()

In [ ]:
#재빈님 코드드
import os
from openai import AzureOpenAI
import gradio as gr
from azure.cognitiveservices.speech import SpeechConfig, SpeechSynthesizer, AudioConfig

# Azure OpenAI 및 Cognitive Search 설정
endpoint = os.getenv("ENDPOINT_URL", "https://team10-eighti.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
search_endpoint = os.getenv("SEARCH_ENDPOINT", "https://team10-eighti-search.search.windows.net")
search_key = os.getenv("SEARCH_KEY", "wnalAsW6FqKRHIR6S3sUZGzNH28Lf3sBOS2ubCZsZxAzSeA205k3")
search_index = os.getenv("SEARCH_INDEX_NAME", "sign-index")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "1BajXTI5Mp0tKrs46XFGuOWjSPXKzOZSKy8e6R3qha1SVQ4lz1PFJQQJ99ALACYeBjFXJ3w3AAABACOGgPvw")

# Azure Speech Service 설정 (한국어 음성 출력)
speech_key = os.getenv("AZURE_SPEECH_KEY", "AwjVcsBAkpnrMNwkobsgJ4SSroO1GkAztrEIYp1JuMTcuKcfDR3wJQQJ99ALACYeBjFXJ3w3AAAYACOGZMDc")
speech_region = os.getenv("AZURE_SPEECH_REGION", "eastus")
speech_config = SpeechConfig(subscription=speech_key, region=speech_region)
speech_config.speech_synthesis_language = "ko-KR"  # 한국어 설정
speech_config.speech_synthesis_voice_name = "ko-KR-SunHiNeural"  # 한국어 음성
speech_synthesizer = SpeechSynthesizer(speech_config=speech_config)

# Azure OpenAI 클라이언트 초기화
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2024-05-01-preview",
)

# 채팅 함수 정의
def chat_with_openai(user_input, chat_history):
    try:
        # 대화 기록 포함 메시지 구성
        messages = [{"role": "system", "content": "너는 수화를 알려주는 전문가야"}]
        for user_msg, assistant_msg in chat_history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})
        messages.append({"role": "user", "content": user_input})

        # Azure OpenAI API 호출
        completion = client.chat.completions.create(
            model=deployment,
            messages=messages,
            max_tokens=800,
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            stream=False,
            extra_body={
                "data_sources": [{
                    "type": "azure_search",
                    "parameters": {
                        "endpoint": f"{search_endpoint}",
                        "index_name": search_index,
                        "semantic_configuration": "sign-semantic",
                        "query_type": "semantic",
                        "fields_mapping": {},
                        "in_scope": True,
                        "role_information": "너는 수화를 알려주는 전문가야",
                        "filter": None,
                        "strictness": 3,
                        "top_n_documents": 5,
                        "authentication": {
                            "type": "api_key",
                            "key": f"{search_key}"
                        }
                    }
                }]
            }
        )

        # 응답 추출 및 대화 기록 업데이트
        assistant_reply = completion.choices[0].message.content.replace(' [doc1]', '').strip()  # 속성 접근 방식으로 수정
        chat_history.append((user_input, assistant_reply))  # 튜플 형태로 추가

        citations = completion.choices[0].message.context['citations'][0]['content'].split('\n')
        video_url = citations[-2]
        image_urls = citations[-3].split(' ')   # 이미지 URL이 여러 개일 수 있음

        # Azure Speech Service로 응답 읽기 (텍스트를 한국어 음성으로 변환)
        # speech_synthesizer.speak_text_async(assistant_reply)

        return chat_history, chat_history, video_url, image_urls

    except Exception as e:
        return [("Error", str(e))], chat_history

# Gradio 인터페이스 구성
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            gr.Markdown("# Azure OpenAI + Cognitive Search + Speech 기반 수화 챗봇")
            chatbot = gr.Chatbot()
            user_input = gr.Textbox(label="Your Message", placeholder="메시지를 입력하세요...")
            clear_button = gr.Button("Clear Chat")
        with gr.Column():
            video_display = gr.Video(label="수어 영상")
            image_display = gr.Gallery(label="수어 이미지", columns=3)

    # 대화 기록 저장
    state = gr.State([])

    # 이벤트 연결    
    user_input.submit(chat_with_openai, [user_input, state], [chatbot, state, video_display, image_display])
    clear_button.click(lambda: [], None, chatbot)
    
# 실행
demo.launch()

In [8]:
import os
import gradio as gr
from openai import AzureOpenAI
from azure.cognitiveservices.speech import SpeechConfig, SpeechSynthesizer
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
import re
import cv2
from PIL import Image
import numpy as np
import io

# Azure OpenAI 및 Cognitive Search 설정
endpoint = os.getenv("ENDPOINT_URL", "https://team10-eighti.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
search_endpoint = os.getenv("SEARCH_ENDPOINT", "https://team10-eighti-search.search.windows.net")
search_key = os.getenv("SEARCH_KEY", "wnalAsW6FqKRHIR6S3sUZGzNH28Lf3sBOS2ubCZsZxAzSeA205k3")
search_index = os.getenv("SEARCH_INDEX_NAME", "sign-index")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "1BajXTI5Mp0tKrs46XFGuOWjSPXKzOZSKy8e6R3qha1SVQ4lz1PFJQQJ99ALACYeBjFXJ3w3AAABACOGgPvw")

# Speech Service 설정 (한국어 음성 출력)
speech_key = os.getenv("AZURE_SPEECH_KEY", "AwjVcsBAkpnrMNwkobsgJ4SSroO1GkAztrEIYp1JuMTcuKcfDR3wJQQJ99ALACYeBjFXJ3w3AAAYACOGZMDc")
speech_region = os.getenv("AZURE_SPEECH_REGION", "eastus")
speech_config = SpeechConfig(subscription=speech_key, region=speech_region)
speech_config.speech_synthesis_language = "ko-KR"
speech_config.speech_synthesis_voice_name = "ko-KR-SunHiNeural"
speech_synthesizer = SpeechSynthesizer(speech_config=speech_config)

# Azure Custom Vision 설정
custom_vision_endpoint = "https://team10eighticustomvision-prediction.cognitiveservices.azure.com/"
prediction_key = "9FRZbiwBubFIcSZ1k88tCTCskOAZwMMAMvnFLuVJ26tlU0V0fsqjJQQJ99ALACYeBjFXJ3w3AAAIACOGOj1S"
project_id = "4218ecac-688a-422b-9e14-2726b938f67c"
published_name = "Iteration8"

credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(endpoint=custom_vision_endpoint, credentials=credentials)

# OpenAI 클라이언트 초기화
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2024-05-01-preview",
)

# 링크 추출 함수
def extract_links(text):
    image_links = []
    video_links = []
    for line in text.split("\n"):
        if line.endswith(".jpg") or line.endswith(".png"):
            image_links.append(line.strip())
        elif line.endswith(".mp4"):
            video_links.append(line.strip())
    return image_links, video_links

# Azure Custom Vision: 프레임 처리 함수
def process_frame(frame):
    if frame is None:
        return None

    # Convert frame to PIL Image for Azure API
    pil_image = Image.fromarray(frame)
    img_byte_arr = io.BytesIO()
    pil_image.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()

    try:
        # Get predictions from Azure Custom Vision
        results = predictor.detect_image(project_id, published_name, img_byte_arr)
        
        # Draw bounding boxes on the frame
        for prediction in results.predictions:
            if prediction.probability > 0.5:  # Threshold for displaying predictions
                box = prediction.bounding_box
                left = int(box.left * frame.shape[1])
                top = int(box.top * frame.shape[0])
                width = int(box.width * frame.shape[1])
                height = int(box.height * frame.shape[0])
                
                # Draw rectangle and label on the frame
                cv2.rectangle(frame, (left, top), (left + width, top + height), (255, 0, 0), 2)
                cv2.putText(frame, f"{prediction.tag_name}: {prediction.probability:.2f}",
                            (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        
        return frame

    except Exception as e:
        print(f"Error during prediction: {e}")
        return frame

# 채팅 함수 정의
def chat_with_openai(user_input, chat_history):
    try:
        # 대화 기록 포함 메시지 구성
        messages = [{"role": "system", "content": "너는 수화를 알려주는 전문가야"}]
        for user_msg, assistant_msg in chat_history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})
        messages.append({"role": "user", "content": user_input})

        # Azure OpenAI API 호출
        completion = client.chat.completions.create(
            model=deployment,
            messages=messages,
            max_tokens=800,
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            stream=False,
            extra_body={
                "data_sources": [{
                    "type": "azure_search",
                    "parameters": {
                        "endpoint": search_endpoint,
                        "index_name": search_index,
                        "semantic_configuration": "sign-semantic",
                        "query_type": "semantic",
                        "fields_mapping": {},
                        "in_scope": True,
                        "role_information": "너는 수화를 알려주는 전문가야",
                        "filter": None,
                        "strictness": 3,
                        "top_n_documents": 5,
                        "authentication": {
                            "type": "api_key",
                            "key": search_key
                        }
                    }
                }]
            }
        )

        # 응답 추출 및 대화 기록 업데이트
        assistant_reply = completion.choices[0].message.context['citations'][0]['content']

        # 이미지 및 동영상 링크 추출
        image_links, video_links = extract_links(assistant_reply)

        chat_history.append((user_input, assistant_reply))

        # 음성 출력 (링크 제외 텍스트만 읽기)
        speech_text = re.sub(r"http\S+", "", assistant_reply).strip()
        speech_synthesizer.speak_text_async(speech_text)

        # Webcam Input과 Custom Vision Output 숨김 상태 유지 (정상 작동 시)
        return chat_history, image_links[0] if image_links else None, video_links[0] if video_links else None, gr.update(visible=False), gr.update(visible=False)

    except Exception as e:
        error_message = f"Error: {str(e)}"
        chat_history.append(("Error", error_message))

        # Webcam Input과 Custom Vision Output 표시 (오류 발생 시)
        return chat_history, None, None, gr.update(visible=True), gr.update(visible=True)

# Gradio 인터페이스 구성
with gr.Blocks() as demo:
    gr.Markdown("# Azure OpenAI + Cognitive Search + Speech 기반 수화 챗봇")

    with gr.Row():
        with gr.Column(scale=3):
            chatbot = gr.Chatbot()
            user_input = gr.Textbox(label="Your Message", placeholder="메시지를 입력하세요...")
            clear_button_chatbot = gr.Button("Clear Chatbot")

        with gr.Column(scale=1):
            image_display_chatbot = gr.Image(label="관련 이미지")
            video_display_chatbot = gr.Video(label="관련 동영상")

    with gr.Row(visible=False) as custom_vision_section:  # 기본적으로 숨김 처리된 섹션
        webcam_input_custom_vision = gr.Image(sources="webcam", streaming=True, mirror_webcam=True, label="Webcam Input")
        output_custom_vision = gr.Image(label="Custom Vision Output")

    state_chatbot = gr.State([])

    # 이벤트 연결: 챗봇 기능
    user_input.submit(chat_with_openai,
                      [user_input, state_chatbot],
                      [chatbot, image_display_chatbot, video_display_chatbot, custom_vision_section])

    clear_button_chatbot.click(lambda: ([], None, None), None,
                               [chatbot, image_display_chatbot, video_display_chatbot])

    # 이벤트 연결: Custom Vision 기능 (실시간 분석)
    webcam_input_custom_vision.stream(process_frame,
                                      inputs=webcam_input_custom_vision,
                                      outputs=output_custom_vision)

demo.launch()


c:\Users\USER\AppData\Local\Programs\Python\Python313\Lib\site-packages\gradio\components\chatbot.py:237: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


c:\Users\USER\AppData\Local\Programs\Python\Python313\Lib\site-packages\gradio\blocks.py:1776: UserWarning: A function (chat_with_openai) returned too many output values (needed: 4, returned: 5). Ignoring extra values.
    Output components:
        [chatbot, image, video, row]
    Output values returned:
        [[('수박', '과일\n두 손을 구부려 손끝이 위로 향하게 하여 입 앞에서 좌우로 움직인다.\nhttp://sldict.korean.go.kr/multimedia/multimedia_files/convert/20151221/232087/IMG000226279_700X466.jpg\nhttp://sldict.korean.go.kr/multimedia/multimedia_files/convert/20191011/626519/MOV000253031_700X466.mp4\n수박')], "http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20151221/232087/IMG000226279_700X466.jpg", "http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20191011/626519/MOV000253031_700X466.mp4", {'__type__': 'update', 'visible': False}, {'__type__': 'update', 'visible': False}]
  warnings.warn(
c:\Users\USER\AppData\Local\Programs\Python\Python313\Lib\site-packages\gradio\blocks.py:1776: User